# Failure Case Gallery — Quant Debugging Notebook

**Idea:** automatically collect the **top‑N worst episodes** and visualize “why it was bad” with compact episode cards.

Ranking modes:
- `suggested` (default): rank by `pnl_suggested` (suggestedAction policy)
- `ppo`: rank by `pnl_ppo` (train a small PPO quickly, then rank)

**Artifacts**
- Episode cards: mini charts (price/returns + suggestedAction + agent pos + drawdown)
- Failure pattern breakdown: summary stats comparing worst vs best


## Setup
Deps: `pandas`, `numpy`, `plotly`. For `ppo` ranking: `gymnasium`, `stable-baselines3`.

Install (if needed):
```bash
cd python-sdk
/Users/serg/projects/prod/ai_patterns/.venv/bin/python -m pip install -e .
/Users/serg/projects/prod/ai_patterns/.venv/bin/python -m pip install pandas numpy plotly gymnasium stable-baselines3
```


In [1]:
import os
import json
import gzip
import time
from dataclasses import dataclass
from pathlib import Path
from datetime import datetime, timezone

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from aipricepatterns import Client

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)


## Parameters
Defaults keep runtime reasonable. Increase anchors/episodes for deeper debugging.


In [2]:
BASE_URL = os.getenv("AIPP_BASE_URL", "https://aipricepatterns.com/api/rust")
API_KEY = os.getenv("AIPP_API_KEY")

SYMBOL = os.getenv("AIPP_RL_SYMBOL", "BTCUSDT")
INTERVAL = os.getenv("AIPP_RL_INTERVAL", "1h")

ANCHOR_POINTS = int(os.getenv("AIPP_SWEEP_ANCHORS", "300"))
LOOKBACK_DAYS = int(os.getenv("AIPP_SWEEP_LOOKBACK_DAYS", "120"))
FORECAST_HORIZON = int(os.getenv("AIPP_RL_HORIZON", "24"))
EPISODES_PER_ANCHOR = int(os.getenv("AIPP_SWEEP_EPISODES_PER_ANCHOR", "30"))
MIN_SIMILARITY = float(os.getenv("AIPP_RL_MIN_SIMILARITY", "0.70"))
SAMPLING_STRATEGY = os.getenv("AIPP_RL_SAMPLING_STRATEGY", "uniform")

# ranking: suggested | ppo
RANK_MODE = os.getenv("AIPP_RANK_MODE", "suggested").strip().lower()
TOP_N = int(os.getenv("AIPP_TOP_N", "12"))

# friction model (applied per step when in position)
FEE_PCT = float(os.getenv("AIPP_FEE_PCT", "0.00"))
SLIP_PCT = float(os.getenv("AIPP_SLIP_PCT", "0.00"))
ROUND_TRIP = bool(int(os.getenv("AIPP_ROUND_TRIP", "0")))

# PPO settings used only if RANK_MODE=ppo
PPO_TIMESTEPS = int(os.getenv("AIPP_PPO_TIMESTEPS", "10000"))
PPO_ENVS = int(os.getenv("AIPP_PPO_ENVS", "8"))
SEED = int(os.getenv("AIPP_SEED", "7"))
OBS_SET = os.getenv("AIPP_OBS_SET", "price_log_vol_dd_cumret_similarity")

NOTEBOOK_DIR = Path.cwd()
CACHE_DIR = Path(os.getenv("AIPP_RESEARCH_CACHE_DIR", str(NOTEBOOK_DIR / "_cache")))
CACHE_DIR.mkdir(parents=True, exist_ok=True)
CACHE_PATH = CACHE_DIR / f"10_gallery_eps_{SYMBOL}_{INTERVAL}_{ANCHOR_POINTS}.json.gz"

print("Base URL:", BASE_URL)
print(f"{SYMBOL} {INTERVAL} anchors={ANCHOR_POINTS} lookbackDays={LOOKBACK_DAYS}")
print(f"episodes/anchor={EPISODES_PER_ANCHOR} minSim={MIN_SIMILARITY} horizon={FORECAST_HORIZON}")
print("rank mode:", RANK_MODE, "TOP_N:", TOP_N)
print(f"feePct={FEE_PCT} slipPct={SLIP_PCT} roundTrip={ROUND_TRIP}")
print(f"PPO: timesteps={PPO_TIMESTEPS} envs={PPO_ENVS} OBS_SET={OBS_SET}")
print("cache:", str(CACHE_PATH))


Base URL: https://aipricepatterns.com/api/rust
BTCUSDT 1h anchors=300 lookbackDays=120
episodes/anchor=30 minSim=0.7 horizon=24
rank mode: suggested TOP_N: 12
feePct=0.0 slipPct=0.0 roundTrip=False
PPO: timesteps=10000 envs=8 OBS_SET=price_log_vol_dd_cumret_similarity
cache: /Users/serg/projects/prod/ai_patterns/python-sdk/research/_cache/10_gallery_eps_BTCUSDT_1h_300.json.gz


## Fetch & cache episodes
We cache raw episodes to keep this notebook reproducible and fast on re-run.


In [3]:
def _safe_float(x, default=np.nan):
    try:
        return float(x)
    except Exception:
        return float(default)

def _map_suggested_action_to_pos(x) -> int:
    if x is None:
        return 0
    if isinstance(x, (int, float)):
        v = int(x)
        if v in (-1, 0, 1):
            return v
        if v in (0, 1, 2):
            return 1 if v == 1 else (-1 if v == 2 else 0)
        return 0
    s = str(x).strip().lower()
    if s in ("hold","flat","none","neutral","wait"): return 0
    if s in ("long","buy","bull","up"): return 1
    if s in ("short","sell","bear","down"): return -1
    return 0

def load_or_fetch_episodes() -> list[dict]:
    if CACHE_PATH.exists():
        with gzip.open(CACHE_PATH, "rt", encoding="utf-8") as f:
            data = json.load(f)
        print("loaded cache episodes:", len(data))
        return data

    client = Client(base_url=BASE_URL, api_key=API_KEY)
    now_ms = int(time.time() * 1000)
    start_ms = now_ms - LOOKBACK_DAYS * 24 * 60 * 60 * 1000
    anchors = np.linspace(start_ms, now_ms, num=ANCHOR_POINTS, dtype=np.int64).tolist()
    out = []
    for i, anchor_ts in enumerate(anchors, start=1):
        res = client.get_rl_episodes(
            symbol=SYMBOL,
            interval=INTERVAL,
            anchor_ts=int(anchor_ts),
            forecast_horizon=FORECAST_HORIZON,
            num_episodes=EPISODES_PER_ANCHOR,
            min_similarity=MIN_SIMILARITY,
            include_actions=True,
            reward_type="returns",
            sampling_strategy=SAMPLING_STRATEGY,
        )
        eps = res.get("episodes") if isinstance(res, dict) else None
        if isinstance(eps, list):
            for ep in eps:
                ts = ep.get("transitions")
                if not isinstance(ts, list) or len(ts) < 2:
                    continue
                out.append({
                    "anchorTs": int(anchor_ts),
                    "similarity": _safe_float(ep.get("similarity"), np.nan),
                    "transitions": ts,
                })
        if i % 25 == 0:
            print(f"{i}/{len(anchors)} anchors, episodes={len(out)}")
        time.sleep(0.02)

    out = [e for e in out if np.isfinite(e.get("similarity", np.nan))]
    out.sort(key=lambda e: e["anchorTs"])
    with gzip.open(CACHE_PATH, "wt", encoding="utf-8") as f:
        json.dump(out, f)
    print("wrote cache episodes:", len(out))
    return out

episodes = load_or_fetch_episodes()
len(episodes)


loaded cache episodes: 8994


8994

## Build per-episode metrics
We compute synthetic price, drawdown, volatility, and PnL for suggestedAction baseline.


In [4]:
def build_arrays(ep: dict):
    ts = ep["transitions"]
    rets = []
    sugg = []
    for t in ts:
        if not isinstance(t, dict):
            continue
        rets.append(_safe_float(t.get("ret", t.get("return", 0.0)), 0.0))
        sugg.append(_map_suggested_action_to_pos(t.get("suggestedAction")))
    rets = np.asarray(rets, dtype=np.float32)
    sugg = np.asarray(sugg, dtype=np.int8)
    n = len(rets)
    price = np.ones(n, dtype=np.float32)
    for i in range(1, n):
        price[i] = max(1e-6, price[i-1] * (1.0 + rets[i-1]))
    cumret = np.cumsum(rets).astype(np.float32)
    peak = np.maximum.accumulate(cumret)
    dd = (cumret - peak).astype(np.float32)
    w = 10
    vol = np.zeros(n, dtype=np.float32)
    for i in range(n):
        a = max(0, i - w + 1)
        vol[i] = float(np.std(rets[a:i+1]))
    return {"ret": rets, "price": price, "cumret": cumret, "dd": dd, "vol": vol, "suggested_pos": sugg.astype(np.float32)}

def pnl_for_pos(rets: np.ndarray, pos: np.ndarray) -> float:
    # per-step pnl with optional friction
    cost = (float(FEE_PCT) + float(SLIP_PCT)) / 100.0
    if ROUND_TRIP:
        cost *= 2.0
    pnl = float(np.sum(pos * rets) - np.sum(np.abs(pos) * cost))
    return pnl

rows = []
for i, ep in enumerate(episodes):
    arr = build_arrays(ep)
    steps = min(len(arr["ret"]), FORECAST_HORIZON)
    rets = arr["ret"][:steps]
    sugg = arr["suggested_pos"][:steps]
    pnl_s = pnl_for_pos(rets, sugg)
    rows.append({
        "idx": int(i),
        "anchorTs": int(ep["anchorTs"]),
        "similarity": float(ep.get("similarity", np.nan)),
        "steps": int(steps),
        "pnl_suggested": float(pnl_s),
        "max_dd": float(np.min(arr["dd"][:steps])) if steps else 0.0,
        "avg_vol": float(np.mean(arr["vol"][:steps])) if steps else 0.0,
        "ret_sum": float(np.sum(rets)) if steps else 0.0,
    })

df = pd.DataFrame(rows)
df["anchorDtUtc"] = pd.to_datetime(df["anchorTs"], unit="ms", utc=True)
df.sort_values(["pnl_suggested"]).head()


,idx,anchorTs,similarity,steps,pnl_suggested,max_dd,avg_vol,ret_sum,anchorDtUtc
8748,8748,1765890240207,0.9760,24,0.0,-0.6269,0.119059,-0.3809,2025-12-16 13:04:00.207000+00:00
1775,1775,1757845504421,0.8637,24,0.0,-0.4579,0.152146,-0.1927,2025-09-14 10:25:04.421000+00:00
8744,8744,1765890240207,0.9771,24,0.0,-0.6269,0.120223,-0.1729,2025-12-16 13:04:00.207000+00:00
8917,8917,1766098293719,0.8508,24,0.0,-0.3192,0.139250,0.2980,2025-12-18 22:51:33.719000+00:00
1769,1769,1757810828836,0.9193,24,0.0,-0.4786,0.083217,0.0566,2025-09-14 00:47:08.836000+00:00


## Optional: compute PPO PnL per episode
If `RANK_MODE=ppo`, we train a small PPO on earlier episodes (80% train split) and compute `pnl_ppo` for each episode.


In [5]:
if RANK_MODE != "ppo":
    print("RANK_MODE!=ppo → skipping PPO training/eval")
else:
    try:
        import gymnasium as gym
        from gymnasium import spaces
        from stable_baselines3 import PPO
        from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
    except Exception as e:
        raise ImportError("For RANK_MODE=ppo you need gymnasium + stable-baselines3") from e

    OBS_MAP = {
        "ret_only": ["ret"],
        "price_log": ["price", "log_price"],
        "price_log_vol_dd_cumret": ["price", "log_price", "vol", "dd", "cumret"],
        "price_log_vol_dd_cumret_similarity": ["price", "log_price", "vol", "dd", "cumret", "similarity"],
        "all": ["price", "log_price", "vol", "dd", "cumret", "similarity", "suggested_pos"],
        "suggested_only": ["suggested_pos"],
    }
    obs_features = OBS_MAP.get(OBS_SET)
    if obs_features is None:
        raise ValueError(f"Unknown OBS_SET={OBS_SET}. Options: {sorted(OBS_MAP.keys())}")

    def build_obs_arrays(ep: dict):
        arr = build_arrays(ep)
        price = arr["price"]
        log_price = np.log(price).astype(np.float32)
        cumret = arr["cumret"].astype(np.float32)
        dd = arr["dd"].astype(np.float32)
        vol = arr["vol"].astype(np.float32)
        sim = float(ep.get("similarity", 0.0))
        similarity = np.full(len(arr["ret"]), sim, dtype=np.float32)
        out = dict(arr)
        out.update({"log_price": log_price, "cumret": cumret, "dd": dd, "vol": vol, "similarity": similarity})
        return out

    class EpisodeEnv(gym.Env):
        metadata = {"render_modes": []}
        def __init__(self, episodes_list: list[dict], obs_features: list[str], seed: int = 0):
            super().__init__()
            self.episodes = episodes_list
            self.obs_features = list(obs_features)
            self.rng = np.random.default_rng(seed)
            self.action_space = spaces.Discrete(3)
            self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(len(self.obs_features),), dtype=np.float32)
            self._t = 0
            self._pos = 0
            self._arr = None

        def reset(self, *, seed=None, options=None):
            super().reset(seed=seed)
            ep = self.episodes[int(self.rng.integers(0, len(self.episodes)))]
            self._arr = build_obs_arrays(ep)
            self._t = 0
            self._pos = 0
            return self._obs(), {}

        def _obs(self):
            return np.asarray([float(self._arr[k][self._t]) for k in self.obs_features], dtype=np.float32)

        def step(self, action):
            a = int(action)
            self._pos = 0 if a == 0 else (1 if a == 1 else -1)
            r = float(self._arr["ret"][self._t])
            pnl = float(self._pos) * r
            cost = (float(FEE_PCT) + float(SLIP_PCT)) / 100.0
            if ROUND_TRIP:
                cost *= 2.0
            pnl -= abs(self._pos) * cost
            self._t += 1
            terminated = self._t >= (min(len(self._arr["ret"]), FORECAST_HORIZON) - 1)
            obs = self._obs() if not terminated else np.zeros((len(self.obs_features),), dtype=np.float32)
            return obs, float(pnl), terminated, False, {}

    episodes_sorted = sorted(episodes, key=lambda e: e["anchorTs"])
    cut = int(0.8 * len(episodes_sorted))
    train_eps = episodes_sorted[:cut]

    def make_env(seed_offset=0):
        return EpisodeEnv(train_eps, obs_features, seed=SEED + seed_offset)

    vec = DummyVecEnv([lambda i=i: make_env(i) for i in range(PPO_ENVS)])
    vec = VecNormalize(vec, norm_obs=True, norm_reward=False, clip_obs=10.0)
    model = PPO("MlpPolicy", vec, verbose=0, seed=SEED, n_steps=256, batch_size=256)
    model.learn(total_timesteps=PPO_TIMESTEPS)
    obs_rms = vec.obs_rms

    def ppo_episode_pnl(ep: dict) -> float:
        arr = build_obs_arrays(ep)
        steps = min(len(arr["ret"]), FORECAST_HORIZON)
        pnl = 0.0
        for t in range(steps - 1):
            obs = np.asarray([float(arr[k][t]) for k in obs_features], dtype=np.float32)
            o = obs.reshape((1, -1))
            o = (o - obs_rms.mean) / np.sqrt(obs_rms.var + 1e-8)
            o = np.clip(o, -10.0, 10.0)
            action, _ = model.predict(o, deterministic=True)
            a = int(action[0])
            pos = 0 if a == 0 else (1 if a == 1 else -1)
            r = float(arr["ret"][t])
            pnl += float(pos) * r
            cost = (float(FEE_PCT) + float(SLIP_PCT)) / 100.0
            if ROUND_TRIP:
                cost *= 2.0
            pnl -= abs(pos) * cost
        return float(pnl)

    df["pnl_ppo"] = [ppo_episode_pnl(episodes[i]) for i in df["idx"].tolist()]
    df.sort_values(["pnl_ppo"]).head()


RANK_MODE!=ppo → skipping PPO training/eval


## Pick worst episodes
We choose worst episodes by the selected ranking metric and render “cards”.


In [6]:
metric = "pnl_suggested" if RANK_MODE != "ppo" else "pnl_ppo"
if metric not in df.columns:
    raise KeyError(f"Missing metric {metric}. If you want PPO ranking set AIPP_RANK_MODE=ppo and rerun.")

worst = df.sort_values([metric], ascending=True).head(TOP_N).reset_index(drop=True)
best = df.sort_values([metric], ascending=False).head(TOP_N).reset_index(drop=True)
worst[["idx","anchorDtUtc","similarity","steps",metric,"max_dd","avg_vol","ret_sum"]]


,idx,anchorDtUtc,similarity,steps,pnl_suggested,max_dd,avg_vol,ret_sum
0,8748,2025-12-16 13:04:00.207000+00:00,0.9760,24,0.0,-0.6269,0.119059,-0.3809
1,1775,2025-09-14 10:25:04.421000+00:00,0.8637,24,0.0,-0.4579,0.152146,-0.1927
2,8744,2025-12-16 13:04:00.207000+00:00,0.9771,24,0.0,-0.6269,0.120223,-0.1729
3,8917,2025-12-18 22:51:33.719000+00:00,0.8508,24,0.0,-0.3192,0.139250,0.2980
4,1769,2025-09-14 00:47:08.836000+00:00,0.9193,24,0.0,-0.4786,0.083217,0.0566
5,6820,2025-11-20 20:36:42.749000+00:00,0.9379,24,0.0,-0.3507,0.113402,-0.2299
6,6819,2025-11-20 20:36:42.749000+00:00,0.9384,24,0.0,-0.3192,0.142830,0.2259
7,8870,2025-12-18 03:35:42.548000+00:00,0.8678,24,0.0,-0.5543,0.129052,0.3112
8,3674,2025-10-09 17:14:26.294000+00:00,0.8858,24,0.0,-0.3629,0.124646,0.6971
9,7989,2025-12-06 12:15:50.575000+00:00,0.9826,24,0.0,-1.1279,0.203265,0.2881


## Episode card renderer
Each card shows: price, cumulative returns, drawdown, per-step returns, and positions (suggested + PPO if available).


In [7]:
def card_for_episode(ep: dict, title: str = ""):
    arr = build_arrays(ep)
    steps = min(len(arr["ret"]), FORECAST_HORIZON)
    rets = arr["ret"][:steps]
    price = arr["price"][:steps]
    cumret = arr["cumret"][:steps]
    dd = arr["dd"][:steps]
    sugg = arr["suggested_pos"][:steps]
    pnl_s = pnl_for_pos(rets, sugg)
    sim = float(ep.get("similarity", np.nan))
    anchor_dt = datetime.fromtimestamp(int(ep["anchorTs"]) / 1000, tz=timezone.utc).strftime("%Y-%m-%d %H:%M")

    fig = make_subplots(
        rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.04,
        row_heights=[0.32, 0.22, 0.22, 0.24],
        subplot_titles=("Price", "CumRet & DD", "Returns", "Positions")
    )
    x = np.arange(steps)
    fig.add_trace(go.Scatter(x=x, y=price, mode="lines", name="price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=x, y=cumret, mode="lines", name="cumret"), row=2, col=1)
    fig.add_trace(go.Scatter(x=x, y=dd, mode="lines", name="dd"), row=2, col=1)
    fig.add_trace(go.Bar(x=x, y=rets, name="ret"), row=3, col=1)
    fig.add_trace(go.Scatter(x=x, y=sugg, mode="lines", name="suggested_pos", line=dict(shape="hv")), row=4, col=1)

    # Optional PPO positions (only if you ran the PPO cell above)
    if RANK_MODE == "ppo" and "model" in globals() and "build_obs_arrays" in globals() and "obs_features" in globals() and "obs_rms" in globals():
        try:
            arr2 = build_obs_arrays(ep)
            ppos = []
            for t in range(max(0, steps - 1)):
                obs = np.asarray([float(arr2[k][t]) for k in obs_features], dtype=np.float32)
                o = obs.reshape((1, -1))
                o = (o - obs_rms.mean) / np.sqrt(obs_rms.var + 1e-8)
                o = np.clip(o, -10.0, 10.0)
                action, _ = model.predict(o, deterministic=True)
                a = int(action[0])
                ppos.append(0 if a == 0 else (1 if a == 1 else -1))
            if len(ppos) == 0:
                ppos = np.zeros(steps, dtype=np.float32)
            else:
                ppos = np.asarray(ppos + [ppos[-1]], dtype=np.float32)
            fig.add_trace(go.Scatter(x=x, y=ppos, mode="lines", name="ppo_pos", line=dict(shape="hv")), row=4, col=1)
        except Exception:
            pass

    header = title or f"anchor={anchor_dt}  sim={sim:.3f}  pnl_suggested={pnl_s:+.5f}"
    fig.update_layout(height=720, title=header, showlegend=True)
    fig.update_xaxes(title_text="step", row=4, col=1)
    return fig

# Render the first worst episode as an example
if len(worst):
    ep0 = episodes[int(worst.loc[0, "idx"])]
    card_for_episode(ep0, title=f"WORST #1 by {metric}")


## Gallery: top‑N worst
Run this cell to render the full gallery (can be heavy in the notebook UI).


In [9]:
figs = []
for i in range(len(worst)):
    ep = episodes[int(worst.loc[i, "idx"])]
    figs.append(card_for_episode(ep, title=f"WORST #{i+1} by {metric}: {worst.loc[i, metric]:+.5f}"))

figs  # display list; in VS Code you can click each


[Figure({
     'data': [{'mode': 'lines',
               'name': 'price',
               'type': 'scatter',
               'x': {'bdata': 'AAECAwQFBgcICQoLDA0ODxAREhMUFRYX', 'dtype': 'i1'},
               'xaxis': 'x',
               'y': {'bdata': ('AACAP78OfD9VWpM/pNGfPx2CkD8Ip3' ... '82PwP0NT8R2DU/shgnP+XWKj/ysRo/'),
                     'dtype': 'f4'},
               'yaxis': 'y'},
              {'mode': 'lines',
               'name': 'cumret',
               'type': 'scatter',
               'x': {'bdata': 'AAECAwQFBgcICQoLDA0ODxAREhMUFRYX', 'dtype': 'i1'},
               'xaxis': 'x2',
               'y': {'bdata': ('SFB8vMB9HT4iH3Q+vQUSPuCVMrzkx5' ... '0ivhvrIr789nW+9gZfvsnln75TBcO+'),
                     'dtype': 'f4'},
               'yaxis': 'y2'},
              {'mode': 'lines',
               'name': 'dd',
               'type': 'scatter',
               'x': {'bdata': 'AAECAwQFBgcICQoLDA0ODxAREhMUFRYX', 'dtype': 'i1'},
               'xaxis': 'x2',
               'y': {'

## Failure pattern breakdown
Compare worst vs best by similarity, volatility, drawdown, and aggregate return.


In [10]:
def summarize_group(name: str, d: pd.DataFrame):
    return pd.Series({
        "group": name,
        "n": len(d),
        "avg_similarity": float(d["similarity"].mean()),
        "avg_vol": float(d["avg_vol"].mean()),
        "avg_max_dd": float(d["max_dd"].mean()),
        "avg_ret_sum": float(d["ret_sum"].mean()),
        f"avg_{metric}": float(d[metric].mean()),
    })

summary = pd.DataFrame([summarize_group("worst", worst), summarize_group("best", best)])
summary


,group,n,avg_similarity,avg_vol,avg_max_dd,avg_ret_sum,avg_pnl_suggested
0,worst,12,0.919650,0.134179,-0.612117,0.088425,0.000000
1,best,12,0.952725,5.165982,-27.439790,-0.486565,96.169191


In [11]:
fig = px.scatter(
    df.sample(min(3000, len(df)), random_state=SEED),
    x="similarity",
    y=metric,
    color="avg_vol",
    title=f"All episodes: {metric} vs similarity (color=avg_vol)",
)
fig.update_layout(height=520)
fig


## What to look for (qualitative)
- Repeated failure regime: e.g. high volatility mean-reversion chopping → position flips lose to costs.
- Big drawdown episodes: strategy takes the wrong side of a trend break.
- Low similarity: retrieval is weak → suggested actions are not reliable.

Typical “fix levers” to test next:
- similarity gating (05/06)
- cost/delay stress (07)
- obs feature changes (08)
- multi-asset training (09)
